In [87]:
#### specify path of Julia Code for 2D Models ####
### By default the working directory ./2D_Models ###
path = string( pwd(), "/2D_Models/")

"/home/jakob/Dokumente/Repositories/Modeling_Framework/2D_Models/"

In [135]:
##### include 2D Modelling Framework ####
include(string(path, "./2D_model_library.jl"));


########## OPERATIONAL ##########
##### how to chose the actual velocity? #####

#op_model = "Constant"
#op_model = "Collision_Free_Speed"
#op_model = "RVO"
#op_model = "SocialForce"
op_model = "AV_Model"
#op_model = "AV_IN_Model"
#op_model = "IN_Costfunction"
#op_model = "IN_Model"

#op_model = "Intrusion_Force"

include_op_model(op_model, path)

########## TACTICAL ##########
#### how to chose the preferred velocity? ###

tact_model = "Constant"
#tact_model = "IN_Model"

include_tact_model(tact_model, path)


########## STRATEGIC ##########
### how to chose the desired velocity ###

strat_model = "Constant"
#strat_model = "Towards_Goal"

include_strat_model(strat_model, path)


########## OPTIMIZATION SCHEME ##########
### If optimization of a cost-function is included, how to solve? ###

#optimization_scheme = "Regular_Sampling"
optimization_scheme = "Random_Sampling"

include_optimization_scheme(optimization_scheme, path)


##### which update scheme? #####

update = "Parallel_Update"
#update = "Step_Update"

include_update(update, path)


##### which order? i.e. Acceleration or Velocity based? ####
#order = "first"
order = "second"

include_order(order, path)


##### PERCEPTIONAL MODEL ####
perception_model = "None"
#perception_model = "Only_in_Front"

include_perception_model(perception_model, path)

In [136]:
#### parameters of the agents ####
p = [1.5, 1.0, 2.0, 1.0, 0.3, 0.0, 0.0, 0.1, 1.0, 1.0, 0.0, 0.0, 0.5, 0.0]
# ModelParameter: v_max, v_des, T, T2, l, step_time, τ_A, τ_R, α, β, ζ_h, ζ_v, r, ϕ
p_desc = "v_max, v_des, T, T2, l, step_time, τ_A, τ_R, α, β, ζ_h, ζ_v, r, ϕ"

### of the system ###
N = 3
system_size = (5.0, 5.0) #important for periodic boundaries
#system_size = (35.0, 35.0) #important for periodic boundaries

### of the simulation ###
sim_p = [0.0, 20.0, 0.05, 0.01, 10.0] 
sim_p_desc = "t_relax, t_max, dt_save, dt, r"

### initialize geometry ###
geometrie = create_geometry_single_obstacle((37.5,31.5), 0.3)
#geometrie = Create_Geometry_Bottleneck(0.2, 0.8, 0.1, system_size)

### initialize crowd ###
menge = create_crowd(N, geometrie)

Init_Hom_Parameters!(p, menge)
Init_Hom_Vels!(menge, 1.0)

β, D, x_min, y_min, ϵ = π/2, sqrt(10), 0.0, 0.0, 0.001
#Init_Two_Agents_α!(menge, β, D, x_min, y_min, ϵ)

D, x_min, y_min, ϵ = 3.0, 0.5, 1.5, 0.2
Init_Three_Agents!(menge, D, x_min, y_min, ϵ)

v2, v1 = 0.1, 0.3
T2, T1 = 1.0, 1.0
D, ϵ = 3.0, 0.0
x_min, y_min = 2.5, 2.5

#Init_Overtaking!(menge, D, x_min, y_min, ϵ, v1, v2, T1, T2);

### Sparse Bottleneck ###

#Init_Agent_Bottleneck!(1.9*pi, 4, menge, system_size)

#ϕ1, ϕ2, dist1, dist2 = 2.5*π/2, 3.5*π/2, 4.0, 4.0
#Init_2_Agents_Bottleneck!(ϕ1, ϕ2, dist1, dist2, menge, system_size)

#ϕ1, ϕ2, ϕ3, dist1, dist2, dist3 = 0.0, 3*π/2, 3.2*π/2, 4.0, 4.0, 3.5
#Init_3_Agents_Bottleneck!(ϕ1, ϕ2, ϕ3, dist1, dist2, dist3, menge, system_size)


#ϕ1, ϕ2, ϕ3, ϕ4, dist1, dist2, dist3, dist4 = 1.2*π, π, 3.2*π/2, 0.0, 4.0, 4.0, 3.8, 4.0
#Init_4_Agents_Bottleneck!(ϕ1, ϕ2, ϕ3, ϕ4, dist1, dist2, dist3, dist4, menge, system_size)

In [137]:
positions, headings = Simulate!(menge, geometrie, sim_p[1], sim_p[2], sim_p[3], sim_p[4], sim_p[5], system_size);

In [138]:
using GR

GR.inline("mov")

j = 1

dt_save = sim_p[3]
geometry_x, geometry_y = rewrite_geometry(geometrie)
boundaries_x, boundaries_y = boundaries_rectangle_positions(system_size)

while j*sim_p[3] < sim_p[2]-sim_p[1]
    
        GR.clearws()

        GR.setwindow(-1, 6, -1, 6)
        GR.setviewport(0.0, 1, 0.0, 1)
    
        #plot_goal(goal1)
        #plot_goal(goal2)

        #plot_geometry(geometry_x, geometry_y, 3.6, 20)
        plot_boundaries(boundaries_x, boundaries_y, 0.2, 999)        
        
        plot_agents(positions[j, :], headings[j, :], 3.6, 100, 1.2, 999, 0.1)
                
        model = op_model
        timer = string("time ", round(j*dt_save, digits = 1), "s")
        GR.text(0.7, 0.95, timer)
        GR.text(0.7, 0.9, model)
        
        GR.updatews()
    
        j = j+1
end

In [139]:
GR.show()

GR.HTML("<video autoplay controls><source type=\"video/mp4\" src=\"data:video/mp4;base64,AAAAFGZ0eXBxdCAgAAACAHF0ICAAAAAId2lkZQAI86ttZGF0AAACsQYF//+t3EXpvebZSLeWLNgg2SPu73gyNjQgLSBjb3JlIDE2MyAtIEguMjY0L01QRUctNCBBVkMgY29kZWMgLSBDb3B5bGVmdCAyMDAzLTIwMjEgLSBodHRwOi8vd3d3LnZpZGVvbGFuLm9yZy94MjY0Lmh0bWwgLSBvcHRpb25zOiBjYWJhYz0xIHJlZj0zIGRlYmxvY2s9MTowOjAgYW5hbHlzZT0weDM6MHgxMTMgbWU9aGV4IHN1Ym1lPTcgcHN5PTEgcHN5X3JkPTEuMDA6MC4wMCBtaXhlZF9yZWY9MSBtZV9yYW5nZT0xNiBjaHJvbWFfbWU9MSB0cmVsbGlzPTEgOHg4ZGN0PTEgY3FtPTAgZGVhZHpvbmU9MjEsMTEgZmFzdF9wc2tpcD0xIGNocm9tYV9xcF9vZmZzZXQ9LTIgdGhyZWFkcz0xMiBsb29rYWhlYWRfdGhyZWFkcz0yIHNsaWNlZF90aHJlYWRzPTAgbnI9MCBkZWNpbWF0ZT0xIGludGVybGFjZWQ9MCBibHVyYXlfY29tcGF0PTAgY29uc3RyYWluZWRfaW50cmE9MCBiZnJhbWVzPTMgYl9weXJhbWlkPTIgYl9hZGFwdD0xIGJfYmlhcz0wIGRpcmVjdD0xIHdlaWdodGI9MSBvcGVuX2dvcD0wIHdlaWdodHA9MiBrZXlpbnQ9MjUwIGtleWludF9taW49MjQgc2NlbmVjdXQ9NDAgaW50cmFfcmVmcmVzaD0wIHJjX2xvb2thaGVhZD00MCByYz1hYnIgbWJ0cmVlPTEgYml0cmF0ZT00MDAwIHJhdGV0b2w9MS4wIHFjb21wPTAuNjAgcXBtaW49MCBxcG1heD02OSBxcHN0ZXA9NCBpcF9yYXRpbz0xLjQwIGFxPTE6MS4wMACAAAAl8GWIhAAa//731LfMsu1Tb7YLqPeiniZfQ3UlAZuWxO06gAAAAwAAAwAAAwDpVIJjfO4A0Yy0AAGmi/SZxwabwAHQGDGM77Gdn2nqvAAWvZ6FoOJ+m2zddvNGY1PNZxqSA/qFRZmVjZ76nsQkA36vZ1t90oJ7I5RD2VckVMfyHmVJdrl7Q6j0N37upVso8tOotrXFVTjVGkhhoXrdZHzshRDEePDLsoPQjxXXWdPdzOhTyL0NMFdep/kRldWxURL+4zu5YHAGRsBRQOfxkgwNt2cZ/Q8WW5KWwTOAAwkeRIsc/4qZZiyzARb2d7Dcj2Abc6I03wQ9K20p1mJ0tpttY3GecW1+pahjzbhE60xtX/7oWToJUJPpqnUQS9VJMcEm3MrtcS95kjduVGYe3cBjsQ7muzjdxvyWTP0qeSY5cODrNsmXb85rPfYBi1qrP8SGvDYBhnt679EvrOJ2ldPdYo2kLXa/Afa8X8n3X0sl4ckDoXDM8tyxBuqTCR8EcKTjGUMZpXEpkPJ5RJ/V1Oh/mxMrEB3wsiXzlpmChXFtG3hzIcydZtGJE26p4/////42sUJlo2xGuiCjhFiTK+luztOld9Qc/jpud47dZNQJRTpOxYB4XTGcA+w0ZSfH+sajowD05ecqF0sY4YXz6sMzVXmvR96OQVvOEIAcExeL6+37+9WOTu+neRCr0s5MVO6CN8WpnZR1ZqT4Dqwp4LBvO9c5HTWyz6n3y1fpPxPKDl54s4P/I2x3g6qV9xmLELUAQqro6nBV706LIk6OfunvsFTnfE6mQGH72Rf7xiGXfWP7yoQzTCmpKZSUcBcUGsYo7vZSjYAyr0AVr5vB7AHT9/6SXmqR/m0U5ni3I0GgF5ZF6JaqGHp261SY08qVf/ygp0LdMBPZR5DnDUVj4gXhTapFizvLPCrukBlZPCD16WcpOkGB/oxY9YXT5+gErvwwmtneBNMTtydFW0YuoAbSTguTbVp2ghWrO5a5yL1jShsy5Qlo+uInOflOlZyeI/jNuuBG+OcmzOMk1I+pNXJgwDOmoTllFSpkItpAAhsVF/vv2YR/tfgZDvVqXKfxTat3QAB4G0MKdNXEj3zeVNoiFR5nIE1+vY3spL3hAa1JeaFoq6GwqXaZYkFQq+xdZLrsnTEVNf4xwe78zjc+4zQzmkIxIqhsax3atjxmistcmm0a/7XFxMb+PNLSK3NkDYyvS8ykZ3xewoS5owNSpxEz9jRM1vMNg2IwW8pWL0qOJTdITUZwK7T39/Sy3Lk4qFt7cFkUeINLdA0Fnxv3+KMoTjwpYVxCVGqXo1XUNQcan3i0Gd/pCdjeVYEzQuUiaw/EbozlC5fAePIzzqkuYztkuVdTKzds2Dy5qIcorEehTXFE2a9i3RMhmxmAE7RV7tTJfS+HjFDAjeKhVt+7h51yabBbYwli3pMgVnwcuWs6IIQZdD7bN+5VjfXAn0hP1WINnDLVh+cLNaRMbPee4dqBRzjyF7yqoGYle0HV4/olcQIsmEOQF1eu0oKJqj0IVK2tRbGTEgPGXPo4bZY0mmYmwAtpXt36jad+wFj79cRAD5Ovgrnqwl5QfeqbeXlteClutgL9+ezCRjzmfuMGz96+F/Xj0LPSt84Z4MMjESCS4ergfH0il14ntvT8XXr/n3AA2kXQWzVNj6d/hLS7m/W49xGapc2APtINnP/+C3N6J3EO9AQSSzjYdd8f5bb4BjEaF3ymQlhtabRt+6egTvC+EAC0gGoMF+OzJZF0u35lB+NKac4RXfs2AmQDEpxEB/4tO4x/Hn9QB2X/Hvyd8IHY4wn5ARzOys36j28rV//Mooi6cGQ9wfCONwwThrYHDJ+wsjZwjB1xfH/d5SUTD4ATcBeZnp8CP75fs2PzBOYxmjAuBG5h/o2zJfXoNgpTTM2Tf5HBgxkdeS2FCMJ5+gAcdz4pS4VUQDpvelMzX9FAH6MQEFEhVqFk+VCCjBIPfH5jDUQ+HiZhyyZyZcN70ebGIWiu2t8xvvxC02frlFgnZMnSfgT//dpWRo/6rWgu7AVOrZRCH1ADiwLhcBJ3NuSD8n5U53O0UeDczt5KBhmBOo9ukNnCkPmOCYrmEwxDGN0Lymnr0IOQh4yeHIOj9aIbLaZUAw99MRN569VM51rS7aTtiff1UhpjGu4k1L+fOxA4a7X9ZxLXpxqzADuWXaiB18L6xI5M9TRbB69fkwDhn1vm06K4sPW3ooWVbS5wDjP5fmipntlS0ApNlRxY2H+tyj4ADgg7IrprOrbYGzqs44pc4oTsZ79hl42nV3c3IFca1eI/aC9v5dUYGZbpUVHBpJBpcpad+R4r1O40gwl4zqGFxGPpPzQgFZW7JVE77pMoo92Y61+1tc8TI5EazeUj7sCeuwp2Hf/SBUptHZCRwtKjTcwwveGMQBEI3B8BtcT3QRN2ZhWwPy/k5zY8EhzCGu1jEQgbmwI9S0e/N8BxHQmXFiSUpl9zKUx06GRNKjp/F7x5bWPtwQ0G2Ds+gyCeDRPXgEeISNco6AYmzfhkiXQIGjXliwo3Lw1/LVtB6fcwmjQS4JOuvdRbyQtc1mgo2sbePRlSqbuEHedv/gUV2EXbDOvc7O9qlybE9awS0GZWX9WqT/PIvShpOwj3VHDimmIhIR42spVyS923NZsvQ9RVNNKPx0atqIRaydmfPMHyG9O9w0lUJPAaCgBC5yOs6N1lpEI2u3pe6pkNMVPGM832b3P6Pp4H91AOBioaWCjoW+sGyzRVzbgG4JY57kegHX/dBKhE4KqSmVX9smxJfEbzTnNiof+GjSyAy9Pn0SEzuIy6ODUjJ9nwNBkn/WcZIbunuHIxWJBSGOatqaB4GoYVPC8SeXO+9H/mOmBIcX3d9LWA1i77pt+j6m/+aiKocSyc08+aG75Qwkgw2Fn5Es7zuSAFVPMEpGn

In [ ]:
Path = "/home/jakob/Dokumente/Sim_Data/"
dir = "90_CollisionFreeSpeed"
header = Header(p, p_desc, sim_p, sim_p_desc, op_model, tact_model, order, update, N, system_size);

Save_Data!(Path, dir, header, positions)

In [ ]:
samples = 15

for i in 1:samples
    
    Random.seed!()
    Init_Hom_Vels!(menge, 0.0)
    
    β, D, x_min, y_min, ϵ = pi, i/3, 2.5, 5.0, 0.00
    Init_Two_Agents_α!(menge, β, D, x_min, y_min, ϵ)

    
    positions, headings, ttcs = Simulate!(menge, geometrie, sim_p[1], sim_p[2], sim_p[3], sim_p[4], sim_p[5], system_size);
    Save_Data!(Path, dir, header, positions)
    
end

In [ ]:
name = "Agents_Test"
path = "/home/jakob/Dokumente/Renne_Software/UMANS/examples/agents/"

Write_Agents_XML(menge, name, path)